<a href="https://colab.research.google.com/github/CalebEng/Ai-Project/blob/master/Ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
import torch 
from torch import nn
#!pip install --upgrade tiktoken
import tiktoken 